# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [33]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.layers.core import Dropout
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) #GPU 확인

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6648232304179442439
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer() #keras의 Tokenizer로 단어에 인덱스를 붙여 줄 수 있다.
    tokenizer.fit_on_texts(x) #fit_on_texts는 입력에 맞게 내부의 word index를 만든다.
    #fit_on_texts 생성 이후 tokenizer.word_index로 단어와 인덱스 딕셔너리를 가져올 수 있다.
    #Tokenizer는 default로 '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ '를 필터링 한다.
    tokenized_data = tokenizer.texts_to_sequences(x) #문장의 각 단어를 index로 치환한다.
    #fit_on_texts 생성 이후 texts_to_sequences를 써서 index Sequence로 만들 수 있다.
    
    return tokenized_data, tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    #시퀀스를 일괄 처리할 때 각 시퀀스는 동일한 길이가 되어야 한다. 
    #문장의 길이가 각각 다르기 때문에 시퀀스 끝에 패딩을 추가해 길이를 일치시킨다.
    
    return pad_sequences(x, maxlen=length, padding="post")
    #padding은 pre로 시퀀스 앞에 패드, post로 시퀀스 뒤에 패드 설정
    #legnth를 None으로 주면, 가장 긴 시퀀스의 길이에 맞춰 패딩된다.
    
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    #사전 처리 구현. neural network를 구성하는 것이 목표.
    
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    #위에서 구현한 tokenize로 토큰화

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    #위에서 구현한 pad로 패딩
        
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    #sparse_categorical_crossentropy를 사용하려면 3차원이어야 한다.
    #reshape로 차원을 바꾼다. (137861, 21) -> (137861, 21, 1)
    #여기서 그냥 preprocess_y.reshape(preprocess_y.shape, 1)로 하면 오류 난다.
    #shape자체로 넣으면 reshape의 인자는 ((137861, 21), 1)로 되기 때문이다.
    #*preprocess_y.shape로 하면 분해하기 때문에(zip에서 * 넣은 것과 비슷한 듯) 인자는 (137861, 21, 1)로 올바르게 들어간다.

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1] #영어 시퀀스의 최대 길이
max_french_sequence_length = preproc_french_sentences.shape[1] #프랑스어 시퀀스의 최대 길이
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)
#fit_on_texts 생성 이후 tokenizer.word_index로 단어와 인덱스 딕셔너리를 가져올 수 있다.

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    
    #network에서 반환한 결과물은 index 시퀀스 이다. 이를 다시 단어로 바꿔 출력해야 한다.
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    #tokenizer.word_index.items()는 요소가 각 (word, index)인 튜플의 list 반환
    #거기서 각각 index: word인 딕셔너리를 만든다.
    index_to_words[0] = '<PAD>'
    #첫 번째 요소는 패딩으로

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
    #argmax로 인덱스 가져와 딕셔너리에서 값을 추출해 연결

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [12]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 0.001
    
    print(input_shape)
    print(output_sequence_length)
    print(english_vocab_size)
    print(french_vocab_size)

    model = Sequential([
        LSTM(output_sequence_length, input_shape=input_shape[1:], return_sequences=True),
        #french_vocab_size이 unit이 된다(출력). 
        #input_shape는 (137861, 21, 1). input_shape[1:]을 해야 (21, 1)이 된다.
        #return_sequences 마지막 출력 반환 여부
        Dense(french_vocab_size, activation="softmax") #FC 연결. 활성화 함수 softmax.
        
        #LSTM대신 비슷한 GRU모델을 사용할 수 있다.
    ])
    
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy, #손실함수.
                  optimizer=Adam(learning_rate), #최적화
                  metrics=['accuracy']) #정확도를 우선하여 학습
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21, 1)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 21, 21)            1932      
_________________________________________________________________
dense_1 (Dense)              (None, 21, 344)           7568      
Total params: 9,500
Trainable params: 9,500
Non-trainable params: 0
_________________________________________________________________
(137861, 21, 1)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 21, 21)            1932      
_________________________________________________________________
dense_2 (Dense)              (None, 21, 344)           7568      
Total params: 9,500
Trainable params: 9,500
Non-trainable params: 0
_________________________________________________________________
Train on 110288 sa

KeyboardInterrupt: 

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [39]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    #임베딩은 n차원 공간에서 유사한 단어 벡터를 가깝게 표현한다.
    #n은 임베디드 벡터의 크기
    learning_rate = 0.001
    
    print(input_shape)
    print(output_sequence_length)
    print(english_vocab_size)
    print(french_vocab_size)
    
    # TODO: Implement
    model = Sequential([
        Embedding(english_vocab_size, 64, input_length=input_shape[1], input_shape=input_shape[1:]),
        #https://keras.io/layers/embeddings/
        #임베딩 후 LSTM 생성
        LSTM(64, return_sequences=True), #LSTM 모델
        Dense(french_vocab_size, activation="softmax") #FC 연결. 활성화 함수 softmax.
    ])
    
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy, #손실함수.
                  optimizer=Adam(learning_rate), #최적화
                  metrics=['accuracy']) #정확도를 우선하여 학습
    
    return model

tests.test_embed_model(embed_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2])) #(137861, 21)
#simple에서는 input이 다르기 때문에 tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Train the neural network
embedded_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embedded_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embedded_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 21, 64)            12736     
_________________________________________________________________
lstm_60 (LSTM)               (None, 21, 64)            33024     
_________________________________________________________________
dense_43 (Dense)             (None, 21, 344)           22360     
Total params: 68,120
Trainable params: 68,120
Non-trainable params: 0
_________________________________________________________________
(137861, 21)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 21, 64)            12736     
_________________________________________________________________
lstm_61 (LSTM)               (None, 21, 64)            33024     
________________________

InvalidArgumentError: indices[68,0] = 199 is not in [0, 199)
	 [[Node: embedding_28/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_28/embeddings/read, embedding_28/Cast)]]

Caused by op 'embedding_28/Gather', defined at:
  File "/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/anaconda/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 112, in start
    self.asyncio_loop.run_forever(

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [14]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    #일반적인 RNN은 앞으로 들어올 입력 사항을 고려하지 않고 이미 입력된 사항만 고려해 예측한다.
    #Bidirectional RNN은 양 방향의 입력 사항을 고려할 수 있다.
    
    learning_rate = 0.001
    
    print(input_shape)
    print(output_sequence_length)
    print(english_vocab_size)
    print(french_vocab_size)
    
    # TODO: Implement
    model = Sequential([
        Bidirectional(LSTM(output_sequence_length, input_shape=input_shape[1:], return_sequences=True), input_shape=input_shape[1:]),
        #Bidirectional으로 RNN을 wrapping하는 클래스 #simple_model에서 사용한 LSTM과 동일
        #https://keras.io/layers/wrappers/#bidirectional
        LSTM(output_sequence_length, input_shape=input_shape[1:], return_sequences=True),
        #simple_model에서 사용한 LSTM과 동일
        Dense(french_vocab_size, activation="softmax") #FC 연결. 활성화 함수 softmax.
    ])
    
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy, #손실함수.
                  optimizer=Adam(learning_rate), #최적화
                  metrics=['accuracy']) #정확도를 우선하여 학습

    return model

tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2, verbose=2)
#Bidirectional으로 래핑한 것만 빼면 simple_model과 거의 유사.

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21, 1)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 21, 42)            3864      
_________________________________________________________________
lstm_6 (LSTM)                (None, 21, 21)            5376      
_________________________________________________________________
dense_5 (Dense)              (None, 21, 344)           7568      
Total params: 16,808
Trainable params: 16,808
Non-trainable params: 0
_________________________________________________________________
(137861, 21, 1)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 21, 42)            3864      
_________________________________________________________________
lstm_8 (LSTM)                (None, 21, 21)            5376      
__________________

KeyboardInterrupt: 

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    #인코더로 문장의 행렬 표현을 만들고, 디코더는 행렬을 출력으로 변환한다.
    
    # OPTIONAL: Implement
    return None
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [37]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size, dropout_rate=0.2):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.001
    dropout_rate = dropout_rate
    
    print(input_shape)
    print(output_sequence_length)
    print(english_vocab_size)
    print(french_vocab_size)
    
    model = Sequential([        
        Embedding(english_vocab_size, 64, input_length=input_shape[1], input_shape=input_shape[1:]),
        #https://keras.io/layers/embeddings/
        Bidirectional(LSTM(output_sequence_length, input_shape=input_shape[1:], return_sequences=False), input_shape=input_shape[1:]),
        #Bidirectional으로 RNN을 wrapping하는 클래스 #simple_model에서 사용한 LSTM과 동일
        #https://keras.io/layers/wrappers/#bidirectional
        RepeatVector(output_sequence_length),
        #https://discussions.udacity.com/t/capstone-project-model-final-assertion-error/304642/5
        Bidirectional(LSTM(output_sequence_length, input_shape=input_shape[1:], return_sequences=True), input_shape=input_shape[1:]),
        LSTM(output_sequence_length, input_shape=input_shape[1:], return_sequences=True),
        #simple_model에서 사용한 LSTM과 동일
        Dense(french_vocab_size, activation="relu"), #FC연결. 활성화 함수 softmax.
        Dropout(dropout_rate),
        Dense(french_vocab_size * 2, activation="relu"),
        Dropout(dropout_rate), #드랍아웃
        Dense(french_vocab_size, activation="softmax") #FC 연결. 활성화 함수 softmax.
    ])
    
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy, #손실함수.
                  optimizer=Adam(learning_rate), #최적화
                  metrics=['accuracy']) #정확도를 우선하여 학습
    
    return model

tests.test_model_final(model_final)

print('Final Model Loaded')
# TODO: Train the final model

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2])) #(137861, 21)
#simple에서는 input이 다르기 때문에 tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Train the neural network
final_rnn_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 15)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 15, 64)            12736     
_________________________________________________________________
bidirectional_31 (Bidirectio (None, 42)                14448     
_________________________________________________________________
repeat_vector_12 (RepeatVect (None, 21, 42)            0         
_________________________________________________________________
bidirectional_32 (Bidirectio (None, 21, 42)            10752     
_________________________________________________________________
lstm_54 (LSTM)               (None, 21, 21)            5376      
_________________________________________________________________
dense_35 (Dense)             (None, 21, 344)           7568      
_________________________________________________________________
dropout_9 (Dropout)          (None, 21, 344)        

KeyboardInterrupt: 

## Prediction (IMPLEMENTATION)

In [36]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(
        x.shape, 
        y.shape[1], 
        len(x_tk.word_index), 
        len(y_tk.word_index))
    
    model.fit(
        x,
        y, 
        batch_size=1024, 
        epochs=10, 
        validation_split=0.2
    )

    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

(137861, 15)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 15, 64)            12736     
_________________________________________________________________
bidirectional_29 (Bidirectio (None, 42)                14448     
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 21, 42)            0         
_________________________________________________________________
bidirectional_30 (Bidirectio (None, 21, 42)            10752     
_________________________________________________________________
lstm_51 (LSTM)               (None, 21, 21)            5376      
_________________________________________________________________
dense_32 (Dense)             (None, 21, 344)           7568      
_________________________________________________________________
dropout_7 (Dropout)          (None, 21, 344)        

KeyboardInterrupt: 

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [2]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 305996 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?